In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/basantadharala/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [4]:
#Data processing

In [5]:
twitter_data = pd.read_csv('twitter.csv',encoding= 'ISO-8859-1')

In [6]:
twitter_data.shape

(1599999, 6)

In [7]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [8]:
#naming the columns and reading the dataset again
column_names = ['target','id', 'date', 'flag','user','text']
twitter_data = pd.read_csv('twitter.csv', names=column_names, encoding= 'ISO-8859-1')

In [9]:
twitter_data.shape

(1600000, 6)

In [10]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [11]:
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [12]:
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [13]:
twitter_data.replace({'target':{4:1}},inplace=True)

In [14]:
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [15]:
# 0--> negative tweet
# 1--> positive tweet

In [16]:
#Stemming 
ps  = PorterStemmer()

In [17]:
def stemming(content):
    
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)

    return stemmed_content

In [18]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [19]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [20]:
X = twitter_data['stemmed_content'].values
y = twitter_data['target'].values

In [21]:
X

array(['switchfoot http twitpic com zl awww bummer shoulda got david carr third day',
       'upset updat facebook text might cri result school today also blah',
       'kenichan dive mani time ball manag save rest go bound', ...,
       'readi mojo makeov ask detail',
       'happi th birthday boo alll time tupac amaru shakur',
       'happi charitytuesday thenspcc sparkschar speakinguph h'],
      dtype=object)

In [22]:
y

array([0, 0, 0, ..., 1, 1, 1])

In [74]:
X_train, X_test , y_train ,y_test = train_test_split(X,y, test_size=0.2,random_state=2, stratify=y)

In [75]:
X_train

array(['watch saw iv drink lil wine', 'hatermagazin',
       'even though favourit drink think vodka coke wipe mind time think im gonna find new drink',
       ..., 'eager monday afternoon',
       'hope everyon mother great day wait hear guy store tomorrow',
       'love wake folger bad voic deeper'], dtype=object)

In [76]:
y_train

array([1, 1, 0, ..., 1, 1, 0])

In [77]:
#converting textual data to numeric data

vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [44]:
X_train.shape

(1280000, 461062)

In [46]:
print(X_train)

  (0, 411579)	0.40645544795883154
  (0, 452823)	0.34084328948050474
  (0, 74849)	0.847714468934378
  (1, 32613)	0.369498671347723
  (1, 343352)	0.33542712922925133
  (1, 25894)	0.3525968653985028
  (1, 216256)	0.36969993787164285
  (1, 92732)	0.6999691273893042
  (2, 294813)	0.2677249229595089
  (2, 452573)	0.3018512423554567
  (2, 226874)	0.2762973951489663
  (2, 445807)	0.1808418494305006
  (2, 406054)	0.15825955961926266
  (2, 240078)	0.19684651785450386
  (2, 309255)	0.34639981742453446
  (2, 133732)	0.25737902281011105
  (2, 311708)	0.4490676957389869
  (2, 237426)	0.20324166748618955
  (2, 40819)	0.4849378955985905
  (3, 445317)	0.3208912441586938
  (3, 235864)	0.5479299482784662
  (3, 305612)	0.772529340026541
  (4, 262199)	0.5855920408046058
  (4, 320351)	0.25693220124474353
  (4, 29281)	0.31911236492563955
  :	:
  (1279997, 275309)	0.20481136364162741
  (1279998, 74882)	0.5470597277057858
  (1279998, 272417)	0.4070050371150132
  (1279998, 298493)	0.3779039371600175
  (1279998,

In [47]:
print(X_test)

  (0, 424400)	0.6117646409601111
  (0, 423023)	0.3844950823043633
  (0, 161746)	0.42222016303726023
  (0, 129876)	0.28280699637652984
  (0, 56232)	0.46867674625479544
  (1, 445807)	0.47981253641606303
  (1, 33965)	0.8773710331997429
  (2, 375261)	0.39484942652414706
  (2, 317606)	0.5444677484633816
  (2, 212869)	0.3799193886022248
  (2, 79027)	0.5096838896350976
  (2, 48457)	0.37885669067249883
  (3, 400050)	0.5159649783215938
  (3, 267525)	0.36693178188923525
  (3, 159160)	0.42965732854021144
  (3, 17457)	0.6438445376151581
  (4, 454128)	0.2680481537137408
  (4, 356009)	0.5173064666668626
  (4, 345667)	0.3669800167227635
  (4, 335494)	0.30472547197217104
  (4, 320351)	0.2762900772587296
  (4, 220041)	0.3543590953022932
  (4, 124527)	0.36159454248878586
  (4, 93622)	0.16417605064285187
  (4, 23364)	0.27092673717325455
  :	:
  (319996, 301030)	0.2684582847276306
  (319996, 151492)	0.19457609824977737
  (319996, 131905)	0.24810725583523013
  (319996, 52444)	0.40803663872258544
  (319996,

In [83]:
model = LogisticRegression(max_iter=10000)

In [84]:
model.fit(X_train,y_train)

LogisticRegression(max_iter=10000)

In [85]:
y_train_pred = model.predict(X_train)

In [86]:
accuracy_score(y_train,y_train_pred)

0.81018125

In [87]:
y_test_pred = model.predict(X_test)

In [88]:
accuracy_score(y_test,y_test_pred)

0.777996875

In [89]:
X_new = X_test[32753]
print(y_test[32753])

prediction = model.predict(X_new)
print(prediction)

if(prediction[0] == 1):
    print('Positive tweet')
else:
    print('Negative tweet')

0
[0]
Negative tweet


In [94]:
from sklearn.tree import DecisionTreeClassifier
dec = DecisionTreeClassifier()

In [ ]:
dec.fit(X_train,y_train)

In [ ]:
y_train_pred1 = dec.predict(X_train)
accuracy_score(y_train,y_train_pred1)

In [ ]:
y_test_pred1 = dec.predict(X_test)
accuracy_score(y_test,y_test_pred1)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
knn.fit(X_train, y_train)

In [ ]:
y_train_pred2 = knn.predict(X_train)
accuracy_score(y_train,y_train_pred2)

In [ ]:
y_test_pred2 = knn.predict(X_test)
accuracy_score(y_test,y_test_pred2)

In [37]:
import pickle
pickle.dump(model, open('model.pkl', 'wb'))

In [38]:
pickle.dump(vectorizer, open('vectorizer.pkl', 'wb'))